In [1]:
# -*- coding: utf-8 -*-

'''
@Author   :   Corley Tang
@contact  :   cutercorleytd@gmail.com
@Github   :   https://github.com/corleytd
@Time     :   2023-01-18 11:07
@Project  :   Hands-on Deep Learning with PyTorch-loss_function_of_neural_network
神经网络的损失函数
'''

# 导入所需的库
import torch
from torch import nn
from torch.nn import functional as F

# 设置随机种子
torch.manual_seed(20230118)

前面已经实现了从0搭建一个神经网络的过程，并且完成了正向传播，得到了模型的输出，接下来需要完成模型的训练即学习过程，让模型的输出尽可能接近真实标签。
## 1.机器学习中的优化思想
之前，的过程虽然已经得到了模型的输出，但是并不能让神经网络的输出结果与真实值尽可能接近。以线性回归为例，其核心是构造一个预测函数来你和特征矩阵X与标签y之间的线性关系，即找出模型的权重w和b，使得线性回归的输出结果与真实值接近。神经网络也是一样，核心人物就是求得一组最合适的权重，使得神经网络的输出结果与真实值尽可能接近，这个找出权重w和b的过程就是**学习**，也称**训练**或**建模**。
**模型的目标**：求解一组最适合的权重向量 ，令神经网络的输出结果与真实值尽量接近。
用来评价w和b是否合适、输出结果与真实值之间差异的标准，就叫做损失函数。一个模型的训练过程如下：
1. 提出基本模型，明确目标
2. 确定损失函数/目标函数
    定义的评估指标用以衡量模型权重为w的情况下预测结果与真实结果的差异，当真实值与预测值差异越大时，可以认为神经网络学习过程中丢失了许多信息，丢失的这部分被形象地称为**损失**，评估真实值与预测值差异和模型学习过程中产生的损失的函数就称为**损失函数**，在数学上，表示为以需要求解的权重向量w为自变量的函数L(w)：
    - 如果损失函数的值很小，则说明模型预测值与真实值很接近，模型在数据集上表现优异，权重优秀
    - 如果损失函数的值大，则说明模型预测值与真实值差异很大，模型在数据集上表现差劲，权重糟糕

    目标是希望损失函数越小越好，因此将问题转变为求解函数L(w)的最小值所对应的自变量w，求解复杂函数就需要复杂的数学工具，主要有两部分：
    - 将损失函数L(w)转变成凸函数的数学方法，常见的有拉格朗日变换等——深度学习中很少用到
    - 在凸函数上求解L(w)的最小值对应的w的方法，也就是以梯度下降为代表的优化算法
3. 确定适合的优化算法
4. 利用优化算法最小化损失函数，求解最佳权重——训练
## 2.回归：误差平方和SSE
对于回归类神经网络而言，最常见的损失函数是误差平方和SSE（Sum of the Squared Errors），还有全部样本的平均损失MSE（均方误差，Mean squared Error），公式如下：
$$S S E=\sum_{i=1}^{m}\left(z_{i}-\hat{z}_{i}\right)^{2} \\
M S E=\frac{1}{m} \sum_{i=1}^{m}\left(z_{i}-\hat{z}_{i}\right)^{2}$$
PyTorch中MSE损失函数的调用如下。

In [2]:
# 使用MSE损失函数
y = torch.randn(50)
y_hat = torch.randn(50)
criterion = nn.MSELoss()  # 实例化MSE类
loss = criterion(y_hat, y)  # 调用，预测值在前、真实值在后
loss, (y - y_hat).square().sum() / y.size(0)

(tensor(2.2906), tensor(2.2906))

In [3]:
# reduction参数：（1）mean：MSE（默认），（2）sum：SSE，（3）：none：逐元素计算
criterion = nn.MSELoss(reduction='sum')
loss = criterion(y_hat, y)
loss

tensor(114.5323)

In [4]:
criterion = nn.MSELoss(reduction='none')
loss = criterion(y_hat, y)
loss

tensor([1.3855e+00, 7.2827e-01, 3.5886e-01, 2.1885e-01, 1.0606e+00, 6.6579e+00,
        1.3619e-01, 1.7970e+00, 7.2628e+00, 3.5205e-01, 9.6028e-01, 3.8422e+00,
        5.1975e-01, 3.7674e-02, 1.7306e-01, 6.5780e+00, 1.1181e+01, 6.1518e-02,
        2.4976e+00, 1.3350e+00, 6.3254e+00, 5.4917e-01, 1.9191e-01, 3.4457e+00,
        5.3008e+00, 1.1419e+00, 5.1255e-03, 2.4853e-03, 1.2540e-01, 1.3146e-02,
        1.6541e-01, 8.0619e-01, 9.8022e-02, 9.2769e+00, 3.0899e+00, 3.8199e-02,
        1.2461e+00, 7.2920e+00, 1.6648e+00, 9.8241e+00, 1.8780e-02, 1.7243e+00,
        8.4382e-03, 3.7473e+00, 1.4622e-01, 8.1785e-01, 4.6945e-01, 5.6041e+00,
        2.1994e-01, 4.0292e+00])

## 3.二分类交叉熵损失函数
在进行分类任务时，一个很好的评价指标是准确率，但是准确率不能进行求导，而且只关注结果、不关注过程。二分类神经网络常用的损失函数是二分类交叉熵损失函数（Binary Cross EntropyLoss），也叫对数损失。BCE被广泛地使用在任何输出结果是二分类的神经网络中，同时不止限于单层神经网络，还可被拓展到多分类中。在全部样本上的MCE公式如下：
$$L(w)=-\sum_{i=1}^{m}\left(y_{i} * \ln \left(\sigma_{i}\right)+\left(1-y_{i}\right) * \ln \left(1-\sigma_{i}\right)\right)$$
二分类交叉熵损失函数的目标，就是求解出使L(w)最小的w取值，是由极大似然估计推导出来的，其基本方法是寻找相应的权重 ，使得目标事件的发生概率最大，即希望每个样本的分类都正确。为了达成让模型拟合好、损失小的目的，我们每时每刻都希望$P(\hat{y}_{i}\mid{x}_{i}, w)$的值等于1，即每时每刻都在追求$P(\hat{y}_{i}\mid{x}_{i}, w)$的最大值。
### 用Tensor实现BCELoss

In [5]:
# 构造数据
data_size = 3000000  # 样本量
X = torch.rand(data_size, 20)  # 特征张量
w = torch.randn(20, 1)  # 权重
y = torch.randint(2, size=(data_size, 1), dtype=torch.float)  # 标签
X.shape, w.shape, y.shape

(torch.Size([3000000, 20]), torch.Size([20, 1]), torch.Size([3000000, 1]))

In [6]:
# 前向传播
z_hat = X @ w
sigma = z_hat.sigmoid()
sigma.shape

torch.Size([3000000, 1])

In [7]:
# 自定义BCELoss：loss = -(y * ln(σ) + (1 - y) *ln(1 - σ))
loss = -(y * torch.log(sigma) + (1 - y) * torch.log(1 - sigma))  # 计算单个样本的BCE损失
loss.shape

torch.Size([3000000, 1])

In [8]:
# 计算总损失和平均损失
loss_sum = loss.sum()
loss_mean = loss.mean()
loss_sum, loss_mean

(tensor(3830514.), tensor(1.2768))

PyTorch提供了BCE损失的2个类：
- BCELoss中只有交叉熵函数，没有sigmoid层，需要输入sigma与真实标签，且顺序不能变化、sigma必须在前
- BCEWithLogitsLoss内置了Sigmoid函数与交叉熵函数，会自动计算输入值的sigmoid值，需要输入z_hat与真实标签，且顺序不能变化

这两个函数都要求预测值与真实标签的结构（shape）及数据类型必须相同，否则不能正常运行。
BCE有2个损失函数，推荐使用BCEWithLogitsLoss，是因为自己直接调用PyTorch的sigmoid()函数存在精度问题，当数据量变大、数据本身也变大时，BCELoss产生的结果可能有精度的损失，BCEWithLogitsLoss中的实现进行了优化，内置的Sigmoid函数可以让精度问题被缩小（因为将指数运算包含在了内部），以维持算法运行时的稳定性，带来更高的精度、更快的速度，因此在精度和速度要求更高的场景中，尽可能选择BCEWithLogitsLoss，如果需要计算准确率等指标，需要sigma，则必须选择BCELoss。

In [9]:
# 使用BCELoss
criterion = nn.BCELoss()
criterion(sigma, y)

tensor(1.2768)

In [10]:
# 使用BCEWithLogitsLoss
criterion = nn.BCEWithLogitsLoss()
criterion(z_hat, y)  # 3个结果一致

tensor(1.2768)

In [11]:
# reduction参数：none
criterion = nn.BCELoss(reduction='none')  # 单个样本
criterion(sigma, y)

tensor([[2.3507],
        [0.0568],
        [2.2575],
        ...,
        [0.1016],
        [0.4998],
        [0.1642]])

In [12]:
# reduction参数：sum
criterion = nn.BCELoss(reduction='sum')  # 损失综合
criterion(sigma, y)

tensor(3830514.)

In [13]:
# 也可以调用functional模块中的对应计算函数（不建议）
F.binary_cross_entropy(sigma, y), F.binary_cross_entropy_with_logits(z_hat, y)

(tensor(1.2768), tensor(1.2768))

## 4.多分类交叉熵损失函数
### 由二分类推广到多分类
二分类交叉熵损失可以被推广到多分类上，但在实际处理时有所区别，一个主要问题在于多酚分类的真实标签可能是任意整数，无法使用y和(1-y)这样的结构来构建似然函数，要让多分类的标签也可以使用0和1来表示，就需要引入**哑变量**，举例如下：
![multi_classification_one_hot](../assets/multi_classification_one_hot.png)

可以看到，原本的真实标签y是含有1、2、3三个分类的列向量，现在将其变成了标签矩阵，每个样本对应一个向量，使用1标注出样本的真实标签的位置、使用0表示样本的真实标签不是这个标签，此时标签矩阵的结构是和Softmax函数输出的概率矩阵的结构一致，并且一一对应，这就是独热编码（One-Hot Encoding）。最终得到的多酚类交叉熵损失公式如下：
$$L(w)=-\sum_{i=1}^{m} y_{i(k=j)} \ln \sigma_{i}$$
可以看到，二分类交叉熵损失其实是多分类交叉熵损失的一种特殊情况。交叉熵函数十分特殊，从计算结果来看，对数操作其实只对Softmax函数的结果σ起效，因此在实际操作中，把这样的函数单独提出来称为LogSoftmax，PyTorch中对应nn.logsoftmax类；同时把对数之外的操作，包括乘标签、加和、取负等过程统称为负对数似然函数（Negative Log Likelihood function），在PyTorch中对应nn.NLLLoss。因此在计算损失函数时，不再需要单独使用Softmax函数。如下：
![cross_entropy_parts](../assets/cross_entropy_parts.png)
### 用PyTorch实现多分类交叉熵损失
用PyTorch实现交叉熵函数时，有2种方式。分别如下。

In [14]:
# 1.调用LogSoftmax和NLLLoss实现
# 构造数据
w = torch.randn(20, 3)  # 权重
y = torch.randint(3, size=(data_size,))  # 标签
X.shape, w.shape, y.shape

(torch.Size([3000000, 20]), torch.Size([20, 3]), torch.Size([3000000]))

In [15]:
# 前向传播
z_hat = X @ w
z_hat.shape

torch.Size([3000000, 3])

In [16]:
# LogSoftmax
logsm = nn.LogSoftmax(-1)
log_sigma = logsm(z_hat)
log_sigma

tensor([[-1.8821e-04, -8.8193e+00, -1.0120e+01],
        [-3.5889e-03, -5.7479e+00, -7.8416e+00],
        [-3.0617e-02, -4.2700e+00, -4.1245e+00],
        ...,
        [-3.1070e-02, -3.8467e+00, -4.6839e+00],
        [-2.1988e-03, -6.1772e+00, -9.0269e+00],
        [-1.0573e-01, -3.9636e+00, -2.5091e+00]])

In [17]:
# NLLLoss
criterion = nn.NLLLoss()
loss = criterion(log_sigma, y)
loss

tensor(3.8036)

In [18]:
# 2.直接调用CrossEntropyLoss
criterion = nn.CrossEntropyLoss()
loss = criterion(z_hat, y)
loss  # 两种方式结果一致

tensor(3.8036)

从上面可以看到，无论时二分类还是多分类，PyTorch都提供了包含输出层激活函数和不包含输出层激活函数的类两种选择，在实际神经网络建模中，类可以被放入定义好的Model类中去构建神经网络的结构，是否包含激活函数由用户根据需要自行选择：
- 重视展示网络结构和灵活性，应该使用不包含输出层激活函数的类
- 重视稳定性和运算精度，使用包含输出层激活函数的类